In [5]:
"""
Assign row,col labels to yEd GraphML nodes based on their x/y positions.

Algorithm (per your spec):
- Group nodes into columns using 1D greedy clustering on x (tolerates small jitters).
- For each column:
  - Determine the "last row" (bottom-most index) as:
      max(existing row labels in that column), else
      median of last-rows across other columns with labels, else
      number of nodes in the column.
  - Estimate the vertical step as the median gap between consecutive y's.
  - Assign rows from bottom→top using a monotone stepper:
      bottom node gets last_row;
      each node above: row = prev_row - max(1, round(dy / y_gap)).
    This handles missing rows (skips numbers) and fuzzy spacing robustly.
- Assign column numbers:
    If a column has consistent existing ",col" labels, keep that number.
    Otherwise assign by left→right order (1..K).
- Write labels back into the existing <y:NodeLabel> text as "row,col" (same field).

Outputs:
- Updated GraphML with new labels.
- Optional CSV report mapping old→new labels.

Usage (script):
    python assign_labels.py input.graphml -o output.graphml -r report.csv

Usage (notebook):
    relabel_graphml("HIMCM_graph.graphml", "HIMCM_graph_assignedNode.graphml", "HIMCM_node_label_report.csv")
"""

import argparse
import math
import re
from collections import Counter, defaultdict
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET

# --- Namespaces ---
NS = {
    "g": "http://graphml.graphdrawing.org/xmlns",
    "y": "http://www.yworks.com/xml/graphml",
}
# Preserve namespace prefixes in the output
ET.register_namespace("", NS["g"])
ET.register_namespace("y", NS["y"])

LABEL_RE = re.compile(r"^\s*(\d+)\s*,\s*(\d+)\s*$")


def _shape_node(node: ET.Element) -> Optional[ET.Element]:
    for d in node.findall("./g:data", NS):
        sn = d.find("./y:ShapeNode", NS)
        if sn is not None:
            return sn
    return None


def _geometry(sn: ET.Element) -> Optional[Tuple[float, float, float, float]]:
    geom = sn.find("./y:Geometry", NS)
    if geom is None:
        return None
    x = float(geom.attrib.get("x", "0"))
    y = float(geom.attrib.get("y", "0"))
    w = float(geom.attrib.get("width", "0"))
    h = float(geom.attrib.get("height", "0"))
    return x, y, w, h


def _get_label(sn: ET.Element) -> Tuple[Optional[ET.Element], str, Optional[int], Optional[int]]:
    lab = sn.find("./y:NodeLabel", NS)
    txt = ""
    row = col = None
    if lab is not None and lab.text:
        txt = lab.text.strip()
        m = LABEL_RE.match(txt)
        if m:
            row, col = int(m.group(1)), int(m.group(2))
    return lab, txt, row, col


def _ensure_label(sn: ET.Element) -> ET.Element:
    lab = sn.find("./y:NodeLabel", NS)
    if lab is None:
        lab = ET.SubElement(sn, f"{{{NS['y']}}}NodeLabel")
        lab.set("visible", "true")
    return lab


def _median_vertical_gap(y_values: List[float]) -> float:
    if len(y_values) < 2:
        return float("nan")
    diffs = np.diff(sorted(y_values))
    diffs = np.abs(diffs)
    return float(np.median(diffs))


def _cluster_columns(df: pd.DataFrame,
                     width_factor: float = 1.5,
                     nn_percentile: float = 5.0,
                     min_eps: float = 8.0) -> Tuple[pd.DataFrame, Dict[int, float]]:
    """Greedy 1D clustering on x-center with adaptive epsilon."""
    sorted_x = np.sort(df["xc"].values)
    x_diffs = np.diff(sorted_x) if len(sorted_x) > 1 else np.array([0.0])
    median_w = float(np.median(df["w"])) if len(df) else 10.0
    eps_candidates = [
        median_w * width_factor,
        (np.percentile(x_diffs, nn_percentile) * 2.0) if len(x_diffs) else median_w * width_factor,
        min_eps,
    ]
    eps = float(max(eps_candidates))

    df_sorted = df.sort_values("xc").reset_index(drop=True)

    col_centers: List[float] = []   # running centers
    col_counts: List[int] = []
    col_ids: List[int] = []

    for _, row in df_sorted.iterrows():
        xcur = row["xc"]
        if col_centers:
            dists = [abs(xcur - c) for c in col_centers]
            j = int(np.argmin(dists))
            if dists[j] <= eps:
                # assign to existing cluster j
                new_center = (col_centers[j] * col_counts[j] + xcur) / (col_counts[j] + 1)
                col_centers[j] = new_center
                col_counts[j] += 1
                col_ids.append(j)
                continue
        # start a new cluster
        col_centers.append(xcur)
        col_counts.append(1)
        col_ids.append(len(col_centers) - 1)

    df_sorted["col_cluster"] = col_ids

    # Map clusters to visual order (left→right) as 1..K
    ordering = sorted([(c, i) for i, c in enumerate(col_centers)], key=lambda t: t[0])
    cluster_to_visual = {cluster_idx: (rank + 1) for rank, (_, cluster_idx) in enumerate(ordering)}
    df_sorted["col_visual"] = df_sorted["col_cluster"].map(cluster_to_visual)

    return df_sorted, {i: c for i, c in enumerate(col_centers)}


def _assign_columns(df: pd.DataFrame, consistency_ratio: float = 0.6) -> Tuple[pd.DataFrame, Dict[int, int]]:
    """Prefer existing ',col' labels if consistent; else use visual order."""
    cluster_final_col: Dict[int, int] = {}
    for cluster_idx, sub in df.groupby("col_cluster"):
        known_cols = [int(c) for c in sub["old_col"].dropna().tolist()]
        if known_cols:
            mode_col, cnt = Counter(known_cols).most_common(1)[0]
            if cnt >= max(1, int(consistency_ratio * len(known_cols))):
                cluster_final_col[cluster_idx] = mode_col
                continue
        cluster_final_col[cluster_idx] = int(sub["col_visual"].iloc[0])
    df["assigned_col"] = df["col_cluster"].map(cluster_final_col)
    return df, cluster_final_col


def _compute_last_rows_and_gaps(df: pd.DataFrame) -> Tuple[Dict[int, int], Dict[int, float], Dict[int, float]]:
    """Collect last_row (bottom-most index), bottom y, and y_gap per column."""
    # Last row per column from known labels
    last_rows_known: Dict[int, int] = {}
    for cluster_idx, sub in df.groupby("col_cluster"):
        known_rows = [int(r) for r in sub["old_row"].dropna().tolist()]
        if known_rows:
            last_rows_known[cluster_idx] = max(known_rows)

    # Fallback last-row: median across known columns (if any), else number of nodes in that column
    last_row_global_fallback = int(np.median(list(last_rows_known.values()))) if last_rows_known else None

    # Global gap fallback
    global_gaps = []
    for _, sub in df.groupby("col_cluster"):
        gap = _median_vertical_gap(sub["yc"].tolist())
        if not math.isnan(gap):
            global_gaps.append(gap)
    global_gap_fallback = float(np.median(global_gaps)) if global_gaps else 20.0

    y_bottom: Dict[int, float] = {}
    y_gap: Dict[int, float] = {}
    last_row: Dict[int, int] = {}

    for cluster_idx, sub in df.groupby("col_cluster"):
        ys = sub["yc"].tolist()
        y_bottom[cluster_idx] = max(ys)
        gap = _median_vertical_gap(ys)
        y_gap[cluster_idx] = gap if not math.isnan(gap) and gap >= 1e-6 else global_gap_fallback

        if cluster_idx in last_rows_known:
            last_row[cluster_idx] = int(last_rows_known[cluster_idx])
        elif last_row_global_fallback is not None:
            last_row[cluster_idx] = int(last_row_global_fallback)
        else:
            last_row[cluster_idx] = int(len(ys))  # rough fallback

    return last_row, y_bottom, y_gap


def _assign_rows_monotone(df: pd.DataFrame,
                          last_row: Dict[int, int],
                          y_bottom: Dict[int, float],
                          y_gap: Dict[int, float]) -> pd.DataFrame:
    """Bottom→top monotone row assignment (handles missing rows)."""
    assigned_rows: List[int] = []

    for cluster_idx, sub in df.groupby("col_cluster", sort=False):
        sub_sorted = sub.sort_values("yc", ascending=False)  # bottom (max y) first
        rows_for_sub: Dict[int, int] = {}

        prev_y = None
        prev_row = None
        for idx, r in sub_sorted.iterrows():
            if prev_row is None:
                cur_row = int(last_row[cluster_idx])
            else:
                dy = prev_y - r["yc"]  # positive moving upward
                step = int(round(dy / y_gap[cluster_idx]))
                if step < 1:
                    step = 1
                cur_row = int(prev_row - step)
                if cur_row < 1:
                    cur_row = 1
            rows_for_sub[idx] = cur_row
            prev_y, prev_row = r["yc"], cur_row

        # Write back
        for idx in sub_sorted.index:
            assigned_rows.append((idx, rows_for_sub[idx]))

    # Merge to df in original df order (df currently is grouped copy; ensure position by index)
    row_map = dict(assigned_rows)
    df = df.copy()
    df["assigned_row"] = df.index.map(lambda i: row_map[i])
    return df

def relabel_graphml(
    input_path: str,
    output_path: str,
    report_csv: Optional[str] = None,
    width_factor: float = 2.0,
    nn_percentile: float = 5.0,
    min_eps: float = 10.0,
    consistency_ratio: float = 0.6,
    print_warnings: bool = True,
) -> None:
    in_path = Path(input_path)
    out_path = Path(output_path)
    assert in_path.exists(), f"Input file not found: {in_path}"

    tree = ET.parse(in_path)
    root = tree.getroot()
    node_elems = root.findall(".//g:graph/g:node", NS)

    rows = []
    for node in node_elems:
        nid = node.attrib.get("id")
        sn = _shape_node(node)
        if sn is None:
            continue
        geom = _geometry(sn)
        if geom is None:
            continue
        x, y, w, h = geom
        xc, yc = x + w / 2.0, y + h / 2.0
        lab_elem, txt, old_row, old_col = _get_label(sn)
        rows.append(
            {"id": nid, "elem": node, "sn": sn, "x": x, "y": y, "w": w, "h": h,
             "xc": xc, "yc": yc, "old_label": txt, "old_row": old_row, "old_col": old_col}
        )

    if not rows:
        raise RuntimeError("No drawable y:ShapeNode nodes with geometry found.")

    df = pd.DataFrame(rows)

    df_sorted, centers = _cluster_columns(df, width_factor, nn_percentile, min_eps)
    df_sorted, cluster_final_col = _assign_columns(df_sorted, consistency_ratio)
    last_row, y_bottom, y_gap = _compute_last_rows_and_gaps(df_sorted)
    df_sorted = _assign_rows_monotone(df_sorted, last_row, y_bottom, y_gap)

    # --- NaN-safe warning checks ---
    def _has_number(x):
        return x is not None and not (isinstance(x, float) and math.isnan(x))

    warnings = []
    for _, r in df_sorted.iterrows():
        if _has_number(r["old_row"]) and int(r["old_row"]) != int(r["assigned_row"]):
            warnings.append(
                f"Seed conflict at node {r['id']}: old_row={int(r['old_row'])} assigned={int(r['assigned_row'])}"
            )
        if _has_number(r["old_col"]) and int(r["old_col"]) != int(r["assigned_col"]):
            warnings.append(
                f"Seed conflict at node {r['id']}: old_col={int(r['old_col'])} assigned={int(r['assigned_col'])}"
            )

    # Write labels back
    for _, r in df_sorted.iterrows():
        sn = r["sn"]
        new_txt = f"{int(r['assigned_row'])},{int(r['assigned_col'])}"
        lab = _ensure_label(sn)
        lab.text = new_txt

    out_path.parent.mkdir(parents=True, exist_ok=True)
    tree.write(out_path, encoding="utf-8", xml_declaration=True)

    if report_csv:
        rcsv = Path(report_csv)
        report = df_sorted[["id", "xc", "yc", "old_label", "assigned_row", "assigned_col"]].copy()
        report["new_label"] = (
            report["assigned_row"].astype(int).astype(str) + "," + report["assigned_col"].astype(int).astype(str)
        )
        report = report.sort_values(["assigned_col", "assigned_row"])[
            ["id", "xc", "yc", "old_label", "new_label"]
        ]
        report.to_csv(rcsv, index=False)

    unique_cols = int(df_sorted["assigned_col"].nunique())
    global_gap_fallback = float(np.median([g for g in y_gap.values()])) if y_gap else float("nan")
    print(f"Columns detected: {unique_cols}")
    print(f"Median vertical gap (per-col medians → global median): {global_gap_fallback:.3f}")
    print(f"GraphML saved to: {out_path}")
    if report_csv:
        print(f"CSV report saved to: {report_csv}")
    if print_warnings and warnings:
        print("Sample warnings (up to 15):")
        for w in warnings[:15]:
            print("  " + w)



In [6]:
relabel_graphml(
    input_path="/home/popsatorn/Desktop/HiMCM_kickoff_2026/code/ArmyKayeeBai/HIMCM_graph_assignedNode_addTrail.graphml",
    output_path="/home/popsatorn/Desktop/HiMCM_kickoff_2026/code/ArmyKayeeBai/HIMCM_graph_final.graphml",
    report_csv="/home/popsatorn/Desktop/HiMCM_kickoff_2026/code/HIMCM_graph_assignedNode.csv"
)

Columns detected: 40
Median vertical gap (per-col medians → global median): 49.288
GraphML saved to: /home/popsatorn/Desktop/HiMCM_kickoff_2026/code/ArmyKayeeBai/HIMCM_graph_final.graphml
CSV report saved to: /home/popsatorn/Desktop/HiMCM_kickoff_2026/code/HIMCM_graph_assignedNode.csv


# Assign Weight

In [9]:
import re
import math
import xml.etree.ElementTree as ET
from pathlib import Path
from typing import Dict, Tuple, Optional, Iterable, Union

# yEd / GraphML namespaces
NS = {
    "g": "http://graphml.graphdrawing.org/xmlns",
    "y": "http://www.yworks.com/xml/graphml",
}

def assign_edge_weights_in_graphml(
    graphml_in: Union[Path, str],
    graphml_out: Union[Path, str],
    *,
    lambda1: float = 1.0,
    lambda2: float = 1.0,
    red_colors: Iterable[str] = ("#ff0000", "#f00"),
    color_prefixes: Iterable[str] = ("#ff0000",),  # treat any hex starting with these as red
    add_aux_fields: bool = True,  # if False, only writes "weight" (and underlying keys)
    axis_tol: float = 1e-3,       # used only as a fallback when labels are missing
) -> None:
    """
    Read a yEd GraphML file and compute per-edge weights, writing ONLY new <data> tags
    under each <edge>. Node labels and edge styles/colors are left untouched.

    Orientation is determined FIRST from the nodes' label text "i,j" (e.g., "14,30").
    - If either i or j stays the same between the two endpoints -> axis (HV)
    - If both i and j change -> diagonal
    If either label is missing/unparsable, we FALL BACK to geometry centers.

    Weight rule (your spec):
      If edge is RED (trail/safe):
          axis:     w_time = 1.0
          diagonal: w_time = sqrt(2)
          w_safety = 1.0
      If edge is DEFAULT/BLACK (harder/less safe):
          axis:     w_time = 1.65
          diagonal: w_time = 1.65 * sqrt(2)
          w_safety = 1.50

      w_total = lambda1 * w_time + lambda2 * w_safety
    """

    # ---------- helpers ----------
    def _get_or_create_key(root: ET.Element, *, for_: str, attr_name: str,
                           attr_type: str, preferred_id: str) -> str:
        # 1) reuse existing key by attr.name & for
        for k in root.findall("g:key", NS):
            if k.attrib.get("for") == for_ and k.attrib.get("attr.name") == attr_name:
                return k.attrib["id"]
        # 2) or by id
        if root.find(f"g:key[@id='{preferred_id}']", NS) is not None:
            return preferred_id
        # 3) create
        k = ET.Element(f"{{{NS['g']}}}key", {
            "id": preferred_id,
            "for": for_,
            "attr.name": attr_name,
            "attr.type": attr_type,
        })
        root.insert(0, k)
        return preferred_id

    def _set_edge_data(edge: ET.Element, key_id: str, text: str) -> None:
        for d in edge.findall("g:data", NS):
            if d.attrib.get("key") == key_id:
                d.text = text
                return
        d = ET.SubElement(edge, f"{{{NS['g']}}}data", {"key": key_id})
        d.text = text

    def _node_centers(root: ET.Element) -> Dict[str, Tuple[float, float]]:
        centers: Dict[str, Tuple[float, float]] = {}
        for n in root.findall(".//g:graph/g:node", NS):
            nid = n.attrib.get("id")
            geom = n.find(".//y:Geometry", NS)
            if geom is None:
                continue
            x = float(geom.attrib.get("x", "0"))
            y = float(geom.attrib.get("y", "0"))
            w = float(geom.attrib.get("width", "0"))
            h = float(geom.attrib.get("height", "0"))
            centers[nid] = (x + w/2.0, y + h/2.0)
        return centers

    def _node_label_coords(root: ET.Element) -> Dict[str, Optional[Tuple[int, int]]]:
        """
        Parse each node's y:NodeLabel text "i,j" into integer coordinates.
        Returns {node_id: (i, j)}; value is None if not present/parsible.
        """
        coord_map: Dict[str, Optional[Tuple[int, int]]] = {}
        pat = re.compile(r"^\s*(-?\d+)\s*,\s*(-?\d+)\s*$")
        for n in root.findall(".//g:graph/g:node", NS):
            nid = n.attrib.get("id")
            txt = None
            # prefer the first NodeLabel that has text
            for d in n.findall("./g:data", NS):
                lab = d.find("./y:ShapeNode/y:NodeLabel", NS)
                if lab is None:
                    lab = d.find("./y:GenericNode/y:NodeLabel", NS)
                if lab is not None and lab.text:
                    txt = lab.text.strip()
                    break
            if txt:
                m = pat.match(txt)
                if m:
                    coord_map[nid] = (int(m.group(1)), int(m.group(2)))
                    continue
            coord_map[nid] = None
        return coord_map

    def _edge_color(edge: ET.Element) -> Optional[str]:
        ls = edge.find(".//y:LineStyle", NS)
        if ls is not None and "color" in ls.attrib:
            return ls.attrib["color"].lower()
        return None  # yEd default (treated as black)

    def _is_red(hexstr: Optional[str]) -> bool:
        if not hexstr:
            return False
        h = hexstr.lower()
        if h in {c.lower() for c in red_colors}:
            return True
        return any(h.startswith(p.lower()) for p in color_prefixes)

    def _orientation_by_label(a: Tuple[int, int], b: Tuple[int, int]) -> str:
        # axis if either coordinate stays the same; diagonal if both change
        return "axis" if (a[0] == b[0] or a[1] == b[1]) else "diagonal"

    def _orientation_by_geometry(p1: Tuple[float, float], p2: Tuple[float, float]) -> str:
        dx = p2[0] - p1[0]
        dy = p2[1] - p1[1]
        return "axis" if (abs(dx) <= axis_tol or abs(dy) <= axis_tol) else "diagonal"

    # ---------- parse ----------
    tree = ET.parse(str(graphml_in))
    root = tree.getroot()
    centers = _node_centers(root)             # fallback
    label_coords = _node_label_coords(root)   # primary

    # keys (create only if needed; never remove/change existing keys)
    k_wtime   = _get_or_create_key(root, for_="edge", attr_name="w_time",   attr_type="double", preferred_id="d_wtime")
    k_wsafety = _get_or_create_key(root, for_="edge", attr_name="w_safety", attr_type="double", preferred_id="d_wsafety")
    k_wtot    = _get_or_create_key(root, for_="edge", attr_name="w_total",  attr_type="double", preferred_id="d_wtot")
    k_weight  = _get_or_create_key(root, for_="edge", attr_name="weight",   attr_type="double", preferred_id="d_weight")

    if add_aux_fields:
        k_kind   = _get_or_create_key(root, for_="edge", attr_name="kind",        attr_type="string", preferred_id="d_kind")
        k_orient = _get_or_create_key(root, for_="edge", attr_name="orientation", attr_type="string", preferred_id="d_orient")
        k_l1     = _get_or_create_key(root, for_="edge", attr_name="lambda1",     attr_type="double", preferred_id="d_lambda1")
        k_l2     = _get_or_create_key(root, for_="edge", attr_name="lambda2",     attr_type="double", preferred_id="d_lambda2")

    # ---------- process edges ----------
    for e in root.findall(".//g:graph/g:edge", NS):
        s = e.attrib.get("source")
        t = e.attrib.get("target")

        # Determine orientation: prefer labels, fallback to geometry
        orient: Optional[str] = None
        a = label_coords.get(s)
        b = label_coords.get(t)
        if a is not None and b is not None:
            orient = _orientation_by_label(a, b)
        elif s in centers and t in centers:
            orient = _orientation_by_geometry(centers[s], centers[t])
        else:
            # Can't classify; skip weighting safely
            continue

        is_trail = _is_red(_edge_color(e))

        # Weights by rule
        if is_trail:
            w_time = 1.0 if orient == "axis" else math.sqrt(2.0)
            w_safety = 1.0
            kind = "trail"
        else:
            w_time = 1.65 if orient == "axis" else 1.65 * math.sqrt(2.0)
            w_safety = 1.50
            kind = "default"

        w_total = lambda1 * w_time + lambda2 * w_safety

        # Write ONLY <data> children under the edge; do not touch style/labels/colors.
        _set_edge_data(e, k_wtime,   f"{w_time:.6f}")
        _set_edge_data(e, k_wsafety, f"{w_safety:.6f}")
        _set_edge_data(e, k_wtot,    f"{w_total:.6f}")
        _set_edge_data(e, k_weight,  f"{w_total:.6f}")  # conventional 'weight' mirror

        if add_aux_fields:
            _set_edge_data(e, k_kind,   kind)
            _set_edge_data(e, k_orient, orient)
            _set_edge_data(e, k_l1,     f"{lambda1:.6f}")
            _set_edge_data(e, k_l2,     f"{lambda2:.6f}")

    # ---------- write ----------
    ET.register_namespace("", NS["g"])
    ET.register_namespace("y", NS["y"])
    ET.ElementTree(root).write(str(graphml_out), encoding="utf-8", xml_declaration=True)


In [10]:
assign_edge_weights_in_graphml(
    graphml_in="../HIMCM_graph_final.graphml",
    graphml_out="../HIMCM_graph_final_addWeight.graphml",
    lambda1=1.0,   # weight on travel-time (w1)
    lambda2=1.0    # weight on safety (w2)
)

# Visualize

In [5]:
"""
Scrollable yEd GraphML viewer (HTML+SVG) that also overlays per-edge weights.

- Positions match yEd: uses y:Geometry (x,y,width,height) directly.
- Y axis direction matches screens (downward), same as yEd.
- Nodes drawn as ellipses (or rects for non-ellipse); labels centered.
- Edges drawn as straight lines between node centers.
- Edge color + width read from y:LineStyle (PolyLineEdge/GenericEdge).
- Edge weights are read from <data> using either attr.name or key id:
      w_time   : 'w_time'   or 'd_wtime'
      w_safety : 'w_safety' or 'd_wsafety'
      w_total  : 'w_total'  or 'd_wtot'
      weight   : 'weight'   or 'd_weight'
- Wraps the large SVG in a fixed-size, scrollable <div>.
- Optional scale factor to make everything bigger.

Usage (notebook or script – example only; do not auto-run here):
    write_scrollable_svg_html(
        graphml_path="HIMCM_graph_final_weighted.graphml",
        out_html="HIMCM_graph_scroll.html",
        scale=1.4,
        viewport_w=1400,
        viewport_h=900,
        show_labels=True,
        show_edges=True,
        edge_label_mode="all",      # 'total' | 'all' | 'none'
        edge_label_decimals=3,
    )
"""

import math
import xml.etree.ElementTree as ET
from pathlib import Path
from typing import Dict, Tuple, List, Optional

# yEd / GraphML namespaces
NS = {
    "g": "http://graphml.graphdrawing.org/xmlns",
    "y": "http://www.yworks.com/xml/graphml",
}

# ------------------------- Utilities -------------------------

def _escape_html(s: str) -> str:
    return (
        s.replace("&", "&amp;")
         .replace("<", "&lt;")
         .replace(">", "&gt;")
         .replace('"', "&quot;")
    )

def _normalize_color(c: Optional[str], default: str = "#888") -> str:
    """Accepts '#RRGGBB', '#RGB', or 'r,g,b' -> '#RRGGBB'. Returns default if invalid/empty."""
    if not c:
        return default
    c = c.strip()
    if c.startswith("#"):
        # Accept #RGB or #RRGGBB (case-insensitive)
        if len(c) in (4, 7):
            return c
        return default
    if "," in c:
        try:
            parts = [int(p.strip()) for p in c.split(",")[:3]]
            r, g, b = (max(0, min(255, v)) for v in parts)
            return f"#{r:02X}{g:02X}{b:02X}"
        except Exception:
            return default
    return c  # fallback; browsers may still accept named colors

def _to_float_maybe(txt: Optional[str]) -> Optional[float]:
    try:
        return float(txt) if txt not in (None, "") else None
    except Exception:
        return None

def _build_edge_keymaps(root: ET.Element):
    """
    Return two maps:
      - name_to_id: attr.name -> key id
      - id_to_name: key id -> attr.name ('' if missing)
    Works even if some keys lack attr.name.
    """
    name_to_id: Dict[str, str] = {}
    id_to_name: Dict[str, str] = {}
    for k in root.findall("g:key", NS):
        if k.attrib.get("for") != "edge":
            continue
        kid = k.attrib.get("id")
        aname = k.attrib.get("attr.name")
        if kid:
            if aname:
                name_to_id[aname] = kid
            id_to_name[kid] = aname or ""
    return name_to_id, id_to_name

def _edge_data_lookup(edge: ET.Element, *, name_to_id: dict,
                      candidates_by_name: List[str], candidates_by_id: List[str]) -> Optional[str]:
    """
    Try to read <data> by attr.name first, then by key id.
    Returns the raw string or None.
    """
    # names -> ids
    possible_ids = [name_to_id[n] for n in candidates_by_name if n in name_to_id]
    # explicit id fallbacks
    possible_ids += candidates_by_id
    for d in edge.findall("g:data", NS):
        if d.attrib.get("key") in possible_ids:
            return (d.text or "").strip() if d.text else None
    return None

# ------------------------- Parsing -------------------------

def _parse_graphml_positions(graphml_path: str):
    tree = ET.parse(graphml_path)
    root = tree.getroot()

    name_to_id, id_to_name = _build_edge_keymaps(root)

    nodes: Dict[str, Dict] = {}
    min_x = float("inf")
    min_y = float("inf")
    max_x = float("-inf")
    max_y = float("-inf")

    # ---- Nodes ----
    for n in root.findall(".//g:graph/g:node", NS):
        nid = n.attrib.get("id")

        # yEd node payload – prefer ShapeNode, fallback GenericNode
        shape = None
        for d in n.findall("./g:data", NS):
            cand = d.find("./y:ShapeNode", NS)
            if cand is None:
                cand = d.find("./y:GenericNode", NS)
            if cand is not None:
                shape = cand
                break
        if shape is None:
            continue

        geom = shape.find("./y:Geometry", NS)
        if geom is None:
            continue

        x = float(geom.attrib.get("x", "0"))
        y = float(geom.attrib.get("y", "0"))
        w = float(geom.attrib.get("width", "0"))
        h = float(geom.attrib.get("height", "0"))
        xc = x + w / 2.0
        yc = y + h / 2.0

        # Colors if present
        fill = "#FFCC00"
        border = "#000000"
        fill_el = shape.find("./y:Fill", NS)
        if fill_el is not None and fill_el.attrib.get("color"):
            fill = _normalize_color(fill_el.attrib["color"], fill)
        border_el = shape.find("./y:BorderStyle", NS)
        if border_el is not None and border_el.attrib.get("color"):
            border = _normalize_color(border_el.attrib["color"], border)

        label_el = shape.find("./y:NodeLabel", NS)
        label = label_el.text.strip() if (label_el is not None and label_el.text) else ""

        # Shape type (default ellipse)
        shp_el = shape.find("./y:Shape", NS)
        shp_type = shp_el.attrib.get("type", "ellipse") if shp_el is not None else "ellipse"

        nodes[nid] = {
            "x": x, "y": y, "w": w, "h": h,
            "xc": xc, "yc": yc,
            "fill": fill, "border": border,
            "label": label,
            "shape": shp_type,
        }

        min_x = min(min_x, x)
        min_y = min(min_y, y)
        max_x = max(max_x, x + w)
        max_y = max(max_y, y + h)

    if not nodes:
        raise RuntimeError("No y:ShapeNode/y:GenericNode with y:Geometry found—check your yEd GraphML.")

    # ---- Edges (color/width + read weights from <data>) ----
    edges: List[Dict] = []
    for e in root.findall(".//g:graph/g:edge", NS):
        s = e.attrib.get("source")
        t = e.attrib.get("target")
        if s not in nodes or t not in nodes:
            continue

        edge_color = "#888"
        edge_width = 1.0

        style_root = None
        for d in e.findall("./g:data", NS):
            cand = d.find("./y:PolyLineEdge", NS)
            if cand is None:
                cand = d.find("./y:GenericEdge", NS)
            if cand is not None:
                style_root = cand
                break

        if style_root is not None:
            ls = style_root.find("./y:LineStyle", NS)
            if ls is not None:
                if ls.attrib.get("color"):
                    edge_color = _normalize_color(ls.attrib["color"], edge_color)
                if ls.attrib.get("width"):
                    try:
                        edge_width = float(ls.attrib["width"])
                    except Exception:
                        pass

        # Robust weight reading: by attr.name or key id
        w_time_txt   = _edge_data_lookup(
            e, name_to_id=name_to_id,
            candidates_by_name=["w_time"],
            candidates_by_id=["d_wtime"]
        )
        w_safety_txt = _edge_data_lookup(
            e, name_to_id=name_to_id,
            candidates_by_name=["w_safety"],
            candidates_by_id=["d_wsafety"]
        )
        w_total_txt  = _edge_data_lookup(
            e, name_to_id=name_to_id,
            candidates_by_name=["w_total", "weight"],   # prefer w_total
            candidates_by_id=["d_wtot", "d_weight"]
        )
        weight_txt   = _edge_data_lookup(
            e, name_to_id=name_to_id,
            candidates_by_name=["weight", "w_total"],   # alternative order
            candidates_by_id=["d_weight", "d_wtot"]
        )

        edges.append({
            "s": s, "t": t,
            "color": edge_color, "width": edge_width,
            "w_time":   _to_float_maybe(w_time_txt),
            "w_safety": _to_float_maybe(w_safety_txt),
            "w_total":  _to_float_maybe(w_total_txt),
            "weight":   _to_float_maybe(weight_txt),
        })

    return nodes, edges, (min_x, min_y, max_x, max_y)

# ------------------------- Renderer -------------------------

def write_scrollable_svg_html(
    graphml_path: str,
    out_html: str,
    scale: float = 1.25,
    viewport_w: int = 1400,
    viewport_h: int = 900,
    show_labels: bool = True,
    show_edges: bool = True,
    margin: int = 20,
    edge_label_mode: str = "total",   # 'total' | 'all' | 'none'
    edge_label_decimals: int = 3,
) -> None:
    """
    edge_label_mode:
      - 'total' : show w_total (fallback to 'weight') at edge midpoint
      - 'all'   : show w_time, w_safety, w_total (fallbacks if missing)
      - 'none'  : no edge labels
    """
    nodes, edges, (min_x, min_y, max_x, max_y) = _parse_graphml_positions(graphml_path)

    # Normalize origin to (margin, margin)
    width = (max_x - min_x) + 2 * margin
    height = (max_y - min_y) + 2 * margin

    def sx(v):  # scale x
        return (v - min_x + margin) * scale
    def sy(v):  # scale y (screen coords: y goes downward like yEd; no inversion)
        return (v - min_y + margin) * scale

    svg_w = width * scale
    svg_h = height * scale

    # Build SVG
    svg_parts: List[str] = []
    svg_parts.append(
        f'<svg xmlns="http://www.w3.org/2000/svg" width="{svg_w:.2f}" height="{svg_h:.2f}" '
        f'viewBox="0 0 {svg_w:.2f} {svg_h:.2f}">'
    )

    # Background grid
    grid_step = max(50, int(100 * scale))
    svg_parts.append(
        f'<defs>'
        f'  <pattern id="grid" width="{grid_step}" height="{grid_step}" patternUnits="userSpaceOnUse">'
        f'    <path d="M {grid_step} 0 L 0 0 0 {grid_step}" fill="none" stroke="#f0f0f0" stroke-width="1"/>'
        f'  </pattern>'
        f'</defs>'
    )
    svg_parts.append(f'<rect x="0" y="0" width="{svg_w:.2f}" height="{svg_h:.2f}" fill="url(#grid)"/>')

    # Edges
    if show_edges and edges:
        for e in edges:
            xs, ys = sx(nodes[e["s"]]["xc"]), sy(nodes[e["s"]]["yc"])
            xt, yt = sx(nodes[e["t"]]["xc"]), sy(nodes[e["t"]]["yc"])
            stroke_w = max(1.0, e["width"] * scale)
            stroke_col = e["color"]

            # draw line
            svg_parts.append(
                f'<line x1="{xs:.2f}" y1="{ys:.2f}" x2="{xt:.2f}" y2="{yt:.2f}" '
                f'stroke="{stroke_col}" stroke-opacity="0.95" stroke-width="{stroke_w:.2f}"/>'
            )

            # Edge weight labels
            if edge_label_mode != "none":
                def fmt(val: Optional[float]) -> str:
                    return f"{val:.{edge_label_decimals}f}" if isinstance(val, float) else "—"

                if edge_label_mode == "total":
                    preferred = e.get("w_total")
                    if preferred is None:
                        preferred = e.get("weight")
                    label_text = fmt(preferred)
                else:  # 'all'
                    t = fmt(e.get("w_time"))
                    s = fmt(e.get("w_safety"))
                    tot = fmt(e.get("w_total") if e.get("w_total") is not None else e.get("weight"))
                    label_text = f"t={t}  s={s}  tot={tot}"

                # Midpoint + rotation so text follows edge direction (upright when steep)
                mx = (xs + xt) / 2.0
                my = (ys + yt) / 2.0
                angle = math.degrees(math.atan2(yt - ys, xt - xs))
                if angle > 90:
                    angle -= 180
                elif angle < -90:
                    angle += 180

                font_px = max(8, int(10 * scale))
                halo_w = max(2, int(2 * scale))
                # White halo behind text for legibility, then the text itself
                svg_parts.append(
                    f'<g transform="rotate({angle:.2f},{mx:.2f},{my:.2f})">'
                    f'  <text x="{mx:.2f}" y="{my:.2f}" font-size="{font_px}px" '
                    f'        text-anchor="middle" dominant-baseline="central" '
                    f'        fill="none" stroke="#fff" stroke-width="{halo_w}">{_escape_html(label_text)}</text>'
                    f'  <text x="{mx:.2f}" y="{my:.2f}" font-size="{font_px}px" '
                    f'        text-anchor="middle" dominant-baseline="central" '
                    f'        fill="#000" opacity="0.9">{_escape_html(label_text)}</text>'
                    f'</g>'
                )

    # Nodes
    for nid, info in nodes.items():
        cx, cy = sx(info["xc"]), sy(info["yc"])
        w, h = info["w"] * scale, info["h"] * scale
        rx, ry = w / 2.0, h / 2.0
        fill = info["fill"]
        stroke = info["border"]

        if info["shape"].lower() == "ellipse":
            svg_parts.append(
                f'<ellipse cx="{cx:.2f}" cy="{cy:.2f}" rx="{rx:.2f}" ry="{ry:.2f}" '
                f'fill="{fill}" stroke="{stroke}" stroke-width="{max(1.0, 1.0*scale):.2f}"/>'
            )
        else:
            x = sx(info["x"]); y = sy(info["y"])
            svg_parts.append(
                f'<rect x="{x:.2f}" y="{y:.2f}" width="{w:.2f}" height="{h:.2f}" '
                f'rx="{2*scale:.2f}" ry="{2*scale:.2f}" '
                f'fill="{fill}" stroke="{stroke}" stroke-width="{max(1.0, 1.0*scale):.2f}"/>'
            )

        if show_labels and info["label"]:
            svg_parts.append(
                f'<text x="{cx:.2f}" y="{cy:.2f}" fill="#000" font-size="{max(8, int(10*scale))}" '
                f'text-anchor="middle" dominant-baseline="middle">{_escape_html(info["label"])}</text>'
            )

    svg_parts.append('</svg>')

    # Wrap in scrollable container
    html = f"""<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<title>{Path(graphml_path).name} — scrollable SVG</title>
<style>
  body {{
    margin: 0;
    font-family: system-ui, -apple-system, Segoe UI, Roboto, sans-serif;
  }}
  .toolbar {{
    padding: 8px 12px;
    background: #fafafa;
    border-bottom: 1px solid #ddd;
    position: sticky;
    top: 0;
    z-index: 2;
  }}
  .viewport {{
    width: {viewport_w}px;
    height: {viewport_h}px;
    overflow: auto;
    border: 1px solid #ccc;
    margin: 10px;
    box-shadow: 0 1px 3px rgba(0,0,0,0.06);
    background: #fff;
  }}
  .hint {{ color: #666; font-size: 12px; }}
</style>
</head>
<body>
  <div class="toolbar">
    <strong>{Path(graphml_path).name}</strong>
    <span class="hint"> | Scroll to pan. Scale={scale:.2f}, Canvas={int(svg_w)}×{int(svg_h)}px.
    Edge labels: {edge_label_mode}</span>
  </div>
  <div class="viewport">
    {''.join(svg_parts)}
  </div>
</body>
</html>
"""

    Path(out_html).write_text(html, encoding="utf-8")


In [6]:
write_scrollable_svg_html(
    graphml_path="../graphs/original/weighted_graph.graphml",
    out_html="../mapGraphs/html/weighted.html",
    scale=1.3,
    viewport_w=1600,
    viewport_h=1000,
    show_labels=True,
    show_edges=True,
)